<a id='top'></a><a name='top'></a>
# Building Machine Learning Pipelines

## Pipeline components

1. [Setup](#setup)
2. Data ingestion
   - [2.1. ExampleGen](#example_gen)
3. Data validation
    - [3.1. StatisticsGen](#statistics_gen)
    - [3.2. SchemaGen](#schema_gen)
    - [3.3. ExampleValidator](#example_validator)
4. Data preprocessing
    - [4.1. Transform](#transform)
5. Model training
    - [5.1. Trainer](#trainer)
6. Model analysis and validation
    - [6.1. Resolver](#resolver)
    - [6.2. Evaluator](#evaluator)
    - [6.3. Pusher](#pusher)
7. Model deployment
    - [7.1 Model Server](#model_server)
    - [7.2. Client Request (simple)](#client_request_simple)
    - [7.3. Client Request (complex)](#client_request_complex)
    - [7.4. Client Request (metadata)](#client_request_metadata)

# 1. Setup
<a id='setup'></a><a name='setup'>
## Imports and Initialization
<a href="#top">[back to top]</a>

In [1]:
import sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Running on Colab. Installing TFX.")
    !pip install tfx &> /dev/null
    !apt-get install tree &> /dev/null
else:
    print("Running locally.")

Running locally.


In [1]:
# Note that importing TFX takes quite a bit of time!
import base64
import docker 
import glob
import os
import requests
import urllib

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propagate = False
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel('ERROR') # DEBUG, INFO, WARN, ERROR, or FATAL

print(f"TFX: {tfx.__version__}")
print(f"TF: {tf.__version__}")

def HR():
    print('-'*40)

HR()
print("Finished imports")

TFX: 1.8.0
TF: 2.8.2
----------------------------------------
Finished imports


In [4]:
# If running on Colab, have to run again, since we restart the runtime
import sys
IN_COLAB = 'google.colab' in sys.modules
print(IN_COLAB)

False


In [5]:
# If need to start with a clean project
!rm -fr tfx_data
!rm -fr tfx_metadata
!rm -fr tfx_pipeline_output
!rm -fr tfx_serving_model
!rm -fr tmp

!ls 
HR()

!tree -I __pycache__

__pycache__                    module.py
download_dataset.py            nohup.out
interactive_pipeline.ipynb     server.log
interactive_pipeline_OLD.ipynb
----------------------------------------
.
├── download_dataset.py
├── interactive_pipeline.ipynb
├── interactive_pipeline_OLD.ipynb
├── module.py
├── nohup.out
└── server.log

0 directories, 6 files


In [6]:
# Setup project structure and artifacts

# Setup pathways for TFX artifact and metadata output
OUTPUT_DIR = '.'
PIPELINE_NAME = 'customer_complaints'
PIPELINE_ROOT = os.path.join(OUTPUT_DIR, 'tfx_pipeline_output', PIPELINE_NAME)
METADATA_PATH = os.path.join(OUTPUT_DIR, 'tfx_metadata', PIPELINE_NAME, 'metadata.db')

# Need absolute path to setup TensorFlow Serving
SERVING_MODEL_DIR = os.path.abspath(os.path.join(OUTPUT_DIR, 'tfx_serving_model', PIPELINE_NAME))

# Setup pathway for project data
DATA_URL = 'http://bit.ly/building-ml-pipelines-dataset'
DATA_ROOT = os.path.join('tfx_data', PIPELINE_NAME)
DATA_FILE = "data.csv"
DATA_PATH = os.path.join(DATA_ROOT, DATA_FILE)

In [7]:
# Download Example data

try:
    os.makedirs(DATA_ROOT, exist_ok=True)
    urllib.request.urlretrieve(DATA_URL, DATA_PATH)
    print(f"Downloaded {DATA_URL} to {DATA_PATH}")
except Exception as e:
    print(f"Error: {e}")

HR()

!tree {DATA_ROOT}

Downloaded http://bit.ly/building-ml-pipelines-dataset to tfx_data/customer_complaints/data.csv
----------------------------------------
tfx_data/customer_complaints
└── data.csv

0 directories, 1 file


In [8]:
# Quick check of data
import pandas as pd

df = pd.read_csv(DATA_PATH)
print(df.isnull().sum())

product                             0
sub_product                     20452
issue                               0
sub_issue                       33868
consumer_complaint_narrative        0
company                             0
state                             186
zip_code                          189
company_response                    0
timely_response                     0
consumer_disputed                   0
dtype: int64


In [9]:
types = list(df.dtypes.unique())
print(types)
HR()

for x in types:
    print(x)
    print(list(df.select_dtypes([x]).columns))
    HR()

[dtype('O'), dtype('int64')]
----------------------------------------
object
['product', 'sub_product', 'issue', 'sub_issue', 'consumer_complaint_narrative', 'company', 'state', 'zip_code', 'company_response', 'timely_response']
----------------------------------------
int64
['consumer_disputed']
----------------------------------------


---
# 2. Create InteractiveContext

In [10]:
# Create an pipeline to use in Jupyter notebook

# TFX interactive context for interactive TFX notebook development.
# https://github.com/tensorflow/tfx/blob/0422c4e4017fcc8a671ded67d8954b4a0f11edf4/tfx/orchestration/experimental/interactive/interactive_context.py#L72

"""
  pipeline_name: Optional name of the pipeline for ML Metadata tracking
    purposes. If not specified, a name will be generated for you.
  pipeline_root: Optional path to the root of the pipeline's outputs. If not
    specified, an ephemeral temporary directory will be created and used.
  metadata_connection_config: Optional metadata_store_pb2.ConnectionConfig
    instance used to configure connection to a ML Metadata connection. If
    not specified, an ephemeral SQLite MLMD connection contained in the
    pipeline_root directory with file name "metadata.sqlite" will be used.
  beam_pipeline_args: Optional Beam pipeline args for beam jobs within
    executor. Executor will use beam DirectRunner as Default.
"""
tfx_metadata=tfx.orchestration.metadata.sqlite_metadata_connection_config(METADATA_PATH)

context = InteractiveContext(
    pipeline_root=PIPELINE_ROOT,
    pipeline_name=PIPELINE_NAME,
    metadata_connection_config=tfx_metadata
)

---
# 3. Data ingestion 
<a id='example_gen'></a>
## 3.1 ExampleGen

<a href="#top">[back to top]</a>

https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/CsvExampleGen

In [11]:
# The trainer ingests data into TFX pipelines.
example_gen = tfx.components.CsvExampleGen(
    input_base=DATA_ROOT
)

# Execute the component interactively.
context.run(example_gen, enable_cache=True)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [12]:
context.show(example_gen)

CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x1531f9fd0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x1531f9a30>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: CsvExampleGen, inputs: {}, outputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={})})

In [13]:
# Examine output artifacts of ExampleGen, training and evaluation examples
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] ./tfx_pipeline_output/customer_complaints/CsvExampleGen/examples/1


In [14]:
# Take a look at training examples

# artifact = example_gen.outputs['examples'].get()[0]
# print(artifact.split_names, artifact.uri)

# Get the URI of the training example output artifact, which is a dir
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')
print(train_uri)
HR()

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
print(tfrecord_filenames)
HR()

# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')
print(dataset)
HR()

# Iterate over the first 3 records and decode them
for tfrecord in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    #pp.pprint(example)
    print(example)
    HR()

./tfx_pipeline_output/customer_complaints/CsvExampleGen/examples/1/Split-train
----------------------------------------
['./tfx_pipeline_output/customer_complaints/CsvExampleGen/examples/1/Split-train/data_tfrecord-00000-of-00001.gz']
----------------------------------------
<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>
----------------------------------------


2022-07-27 20:56:42.114078: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


features {
  feature {
    key: "company"
    value {
      bytes_list {
        value: "Encore Capital Group"
      }
    }
  }
  feature {
    key: "company_response"
    value {
      bytes_list {
        value: "Closed with explanation"
      }
    }
  }
  feature {
    key: "consumer_complaint_narrative"
    value {
      bytes_list {
        value: "I was denied employment because of a judgment against me.  I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC .  Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement.  I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC.  "
      }
    }
  }
  feature {
    key: "consumer_disputed"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "issue"
    value {
      bytes_list {
        value: "Disclosure verification of debt"
      }
    }

In [15]:
!tree {PIPELINE_ROOT}/CsvExampleGen

./tfx_pipeline_output/customer_complaints/CsvExampleGen
└── examples
    └── 1
        ├── Split-eval
        │   └── data_tfrecord-00000-of-00001.gz
        └── Split-train
            └── data_tfrecord-00000-of-00001.gz

4 directories, 2 files


---
# 4. Data validation
<a id='statistics_gen'></a>
## 4.1 StatisticsGen
<a href="#top">[back to top]</a>

https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/StatisticsGen

In [16]:
# The StatisticsGen TFX pipeline component generates features statistics 
# over both training and serving data
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples']
)

context.run(statistics_gen, enable_cache=True)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [17]:
# Visualize the outputted statistics of StatisticsGen
context.show(statistics_gen.outputs['statistics'])

In [18]:
# access the artifact properties
for artifact in statistics_gen.outputs['statistics'].get():
    #print(dir(artifact))
    #HR()
    print(artifact)

Artifact(artifact: id: 2
type_id: 16
uri: "./tfx_pipeline_output/customer_complaints/StatisticsGen/statistics/2"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "statistics"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "StatisticsGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.8.0"
  }
}
state: LIVE
name: "statistics"
, artifact_type: id: 16
name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
base_type: STATISTICS
)


In [19]:
!tree {PIPELINE_ROOT}/StatisticsGen

./tfx_pipeline_output/customer_complaints/StatisticsGen
└── statistics
    └── 2
        ├── Split-eval
        │   └── FeatureStats.pb
        └── Split-train
            └── FeatureStats.pb

4 directories, 2 files


<a id='schema_gen'></a>
## 4.2. SchemaGen
<a href="#top">[back to top]</a>

In [20]:
# The SchemaGen component uses TensorFlow Data Validation to generate a 
# schema from input statistics. The following TFX libraries use the schema:
#   TensorFlow Data Validation
#   TensorFlow Transform
#   TensorFlow Model Analysis
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)

context.run(schema_gen, enable_cache=True)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [21]:
# Show the generated schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'consumer_complaint_narrative',BYTES,required,,-
'consumer_disputed',INT,required,,-
'issue',STRING,required,,'issue'
'product',STRING,required,,'product'
'state',STRING,required,,'state'
'sub_issue',STRING,required,,'sub_issue'
'sub_product',STRING,required,,'sub_product'


,Values
Domain,
'company_response',"'Closed', 'Closed with explanation', 'Closed with monetary relief', 'Closed with non-monetary relief', 'Untimely response'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender damaged or destroyed vehicle', 'Lender repossessed or sold the vehicle', 'Lender sold the property', 'Loan modification,collection,foreclosure', 'Loan servicing, payments, escrow account', 'Lost or stolen check', 'Lost or stolen money order', 'Making/receiving payments, sending money', 'Managing the line of credit', 'Managing the loan or lease', 'Managing, opening, or closing account', 'Money was not available when promised', 'Other', 'Other fee', 'Other service issues', 'Other transaction issues', 'Overdraft, savings or rewards features', 'Overlimit fee', 'Payment to acct not credited', 'Payoff process', 'Privacy', 'Problems caused by my funds being low', 'Problems when you are unable to pay', 'Received a loan I didn\'t apply for', 'Rewards', 'Sale of account', 'Settlement process and costs', 'Shopping for a line of credit', 'Shopping for a loan or lease', 'Taking out the loan or lease', 'Taking/threatening an illegal action', 'Transaction issue', 'Unable to get credit report/credit score', 'Unauthorized transactions/trans. issues', 'Unexpected/Other fees', 'Unsolicited issuance of credit card', 'Using a debit or ATM card', 'Wrong amount charged or received', 'Incorrect exchange rate'"
'product',"'Bank account or service', 'Consumer Loan', 'Credit card', 'Credit reporting', 'Debt collection', 'Money transfers', 'Mortgage', 'Other financial service', 'Payday loan', 'Prepaid card', 'Student loan'"
'state',"'AA', 'AE', 'AK', 'AL', 'AP', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FM', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'"
'sub_issue',"'Account status', 'Account terms', 'Account terms and changes', 'Applied for loan/did not receive money', 'Attempted to collect wrong amount', 'Attempted to/Collected exempt funds', 'Billing dispute', 'Called after sent written cease of comm', 'Called outside of 8am-9pm', 'Can\'t contact lender', 'Can\'t decrease my monthly payments', 'Can\'t get flexible payment options', 'Can\'t qualify for a loan', 'Can\'t stop charges to bank account', 'Can\'t temporarily postpone payments', 'Charged bank 

In [22]:
!tree {PIPELINE_ROOT}/SchemaGen

./tfx_pipeline_output/customer_complaints/SchemaGen
└── schema
    └── 3
        └── schema.pbtxt

2 directories, 1 file


<a id='example_validator'></a>
## 4.3. ExampleValidator
<a href="#top">[back to top]</a>

In [23]:
# The ExampleValidator pipeline component identifies anomalies in 
# training and serving data.
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

context.run(example_validator, enable_cache=True)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [24]:
context.show(example_validator.outputs['anomalies'])

In [25]:
!tree {PIPELINE_ROOT}/ExampleValidator

./tfx_pipeline_output/customer_complaints/ExampleValidator
└── anomalies
    └── 4
        ├── Split-eval
        │   └── SchemaDiff.pb
        └── Split-train
            └── SchemaDiff.pb

4 directories, 2 files


In [26]:
dir(example_validator)

['DRIVER_CLASS',
 'EXECUTOR_SPEC',
 'SPEC_CLASS',
 '_CONSTRUCT_ARGS',
 '_CONSTRUCT_CLS',
 '_CONSTRUCT_KWARGS',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_pip_dependency',
 '_downstream_nodes',
 '_id',
 '_pip_dependencies',
 '_resolve_pip_dependencies',
 '_upstream_nodes',
 '_validate_component_class',
 '_validate_spec',
 'add_downstream_node',
 'add_downstream_nodes',
 'add_upstream_node',
 'add_upstream_nodes',
 'component_id',
 'component_type',
 'downstream_nodes',
 'driver_class',
 'exec_properties',
 'executor_spec',
 'from_json_dict',
 'get_class_type',
 'id',
 'inputs',
 'outputs',
 'platform_config'

In [27]:
example_validator.with_platform_config

<bound method BaseComponent.with_platform_config of ExampleValidator(spec: <tfx.types.standard_component_specs.ExampleValidatorSpec object at 0x157539820>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x157539490>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: ExampleValidator, inputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}), 'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={})}, outputs: {'anomalies': OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={})})>

---
# 5. Data preprocessing
<a id='transform'></a>
## 5.1 Transform

<a href="#top">[back to top]</a>

https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Transform

In [28]:
# Transform and Model
_module_file = './module.py'

In [29]:
%%writefile {_module_file}
# This file written from interactive_pipeline.ipynb

import os
from typing import Union
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_transform as tft

LABEL_KEY = "consumer_disputed"

################
# Transform code
################

# feature name, feature dimensionality
ONE_HOT_FEATURES = {
    "product": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90,
}

# feature name, bucket count
BUCKET_FEATURES = {"zip_code": 10}

# feature name, value is unused
TEXT_FEATURES = {"consumer_complaint_narrative": None}


os.environ["TFHUB_CACHE_DIR"] = "tmp/tfhub"


def transformed_name(key: str) -> str:
    return key + "_xf"


def fill_in_missing(x: Union[tf.Tensor, tf.SparseTensor]) -> tf.Tensor:
    """Replace missing values in a SparseTensor.

    Fills in missing values of `x` with '' or 0, and converts to a
    dense tensor.

    Args:
      x: A `SparseTensor` of rank 2.  Its dense shape should have
        size at most 1 in the second dimension.

    Returns:
      A rank 1 tensor where missing values of `x` have been filled in.
    """
    if isinstance(x, tf.sparse.SparseTensor):
        default_value = "" if x.dtype == tf.string else 0
        x = tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
            default_value,
        )
    return tf.squeeze(x, axis=1)


def convert_num_to_one_hot(label_tensor: tf.Tensor, num_labels: int = 2) -> tf.Tensor:
    """
    Convert a label (0 or 1) into a one-hot vector
    Args:
        int: label_tensor (0 or 1)
    Returns
        label tensor
    """
    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])


def convert_zip_code(zipcode: str) -> tf.float32:
    """
    Convert a zipcode string to int64 representation. In the dataset the
    zipcodes are anonymized by repacing the last 3 digits to XXX. We are
    replacing those characters to 000 to simplify the bucketing later on.

    Args:
        str: zipcode
    Returns:
        zipcode: int64
    """
    zipcode = tf.strings.regex_replace(zipcode, r"X{0,5}", "0")
    zipcode = tf.strings.to_number(zipcode, out_type=tf.float32)
    return zipcode


def preprocessing_fn(inputs: tf.Tensor) -> tf.Tensor:
    """tf.transform's callback function for preprocessing inputs.

    Args:
      inputs: map from feature keys to raw not-yet-transformed features.

    Returns:
      Map from string feature key to transformed feature operations.
    """
    outputs = {}

    for key in ONE_HOT_FEATURES.keys():
        dim = ONE_HOT_FEATURES[key]
        int_value = tft.compute_and_apply_vocabulary(
            fill_in_missing(inputs[key]), top_k=dim + 1
        )
        outputs[transformed_name(key)] = convert_num_to_one_hot(
            int_value, num_labels=dim + 1
        )

    for key, bucket_count in BUCKET_FEATURES.items():
        temp_feature = tft.bucketize(
            convert_zip_code(fill_in_missing(inputs[key])),
            bucket_count,
        )
        outputs[transformed_name(key)] = convert_num_to_one_hot(
            temp_feature, num_labels=bucket_count + 1
        )

    for key in TEXT_FEATURES.keys():
        outputs[transformed_name(key)] = fill_in_missing(inputs[key])

    outputs[transformed_name(LABEL_KEY)] = fill_in_missing(inputs[LABEL_KEY])

    return outputs


################
# Model code
################


def get_model(show_summary: bool = True) -> tf.keras.models.Model:
    """
    This function defines a Keras model and returns the model as a Keras object.
    """

    # One-hot categorical features
    input_features = []
    for key, dim in ONE_HOT_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,), name=transformed_name(key))
        )

    # Adding bucketized features
    for key, dim in BUCKET_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,), name=transformed_name(key))
        )

    # Adding text input features
    input_texts = []
    for key in TEXT_FEATURES.keys():
        input_texts.append(
            tf.keras.Input(shape=(1,), name=transformed_name(key), dtype=tf.string)
        )


    # Embed text features
    MODULE_URL = "https://tfhub.dev/google/universal-sentence-encoder/4"
    embed = hub.KerasLayer(MODULE_URL)
    
    # Keras inputs are two-dimensional, but the encoder expects one-dimensional inputs.
    reshaped_narrative = tf.reshape(input_texts[0], [-1])
    embed_narrative = embed(reshaped_narrative)
    deep_ff = tf.keras.layers.Reshape((512,), input_shape=(1, 512))(embed_narrative)

    deep = tf.keras.layers.Dense(256, activation="relu")(deep_ff)
    deep = tf.keras.layers.Dense(64, activation="relu")(deep)
    deep = tf.keras.layers.Dense(16, activation="relu")(deep)

    wide_ff = tf.keras.layers.concatenate(input_features)
    wide = tf.keras.layers.Dense(16, activation="relu")(wide_ff)

    both = tf.keras.layers.concatenate([deep, wide])

    output = tf.keras.layers.Dense(1, activation="sigmoid")(both)

    inputs = input_features + input_texts

    keras_model = tf.keras.models.Model(inputs, output)
    
    keras_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.TruePositives(),
        ],
    )
    if show_summary:
        keras_model.summary()

    return keras_model


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")



# Applying the preprocessing graph to model inputs.
# This is called in the signature of run_fn().
def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""

    # Load the preprocessing graph.
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        # Parse the raw tf.Example records from the request.
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        # Apply the preprocessing transformation to raw data.
        transformed_features = model.tft_layer(parsed_features)
    
        # Perform prediction with preprocessed data.
        outputs = model(transformed_features)
        return {"outputs": outputs}

    return serve_tf_examples_fn




def _input_fn(file_pattern, tf_transform_output, batch_size=64):
    """Generates features and label for tuning/training.
    Data loading for model training and validation is performed in batches, 
    and the loading is handled by the input_fn() function.

    The input_fn function lets us load the compressed, preprocessed datasets
    that were generated by the previous Transform step.
    To do this, we need to pass the tf_transform_output to the function. 
    This gives us the data schema to load the dataset from the TFRecord data 
    structures generated by the Transform component.
    By using the preprocessed datasets, we can avoid data preprocessing during 
    training and speed up the training process.
    The input_fn returns a generator (a batched_features_dataset) that will supply 
    data to the model one batch at a time.

    Args:
    file_pattern: input tfrecord file pattern.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

      Returns:
        A dataset that contains (features, indices) tuple where features is a
          dictionary of Tensors, and indices is a single Tensor of
          label indices.
    """
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # The dataset will be batched into the correct batch size.
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY),
    )

    return dataset




# TFX Trainer will call this function.
# The run_fn function receives a set of arguments, including the 
# transform graph, example datasets, and training parameters through 
# the fn_args object.

def run_fn(fn_args):
    """Train the model based on given args.

    The run_fn() function is a generic entry point to the training steps 
    and not tf.Keras specific. It carries out the following steps:
    
    1. Loading the training and validation data (or the data generator) 
    2. Defining the model architecture and compiling the model 
    3. Training the model
    4. Exporting the model to be evaluated in the next pipeline step

    This function is fairly generic and could be reused with any other 
    tf.Keras model. The project-specific details are defined in helper 
    functions like get_model() or input_fn().

    Args:
    fn_args: Holds args used to train the model as name/value pairs.
    """
    
    # Load the data
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    # Call the input_fn to get data generators.
    train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 64)
    eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 64)
    
    
    
    
    # Call the get_model function to get the compiled Keras model.
    # strategy, mode = detect_platform()
    # print(strategy)
    # print(mode)
    
    # with strategy.scope():
    #     # Run compile step within distributed strategy
    model = get_model()

    

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )
    callbacks = [tensorboard_callback]

    # Train the model using the number of training and evaluation steps passed by the Trainer component.
    model.fit(
        train_dataset,
        epochs=1,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks,
    )

    # Need to export the model.
    # Combine the preprocessing steps from the previous pipeline component with the trained model 
    # and save the model in TensorFlow’s SavedModel format.
    # Define a model signature based on the graph generated.
    # The run_fn exports the get_serve_tf_examples_fn as part of the model signature.
    # When a model has been exported and deployed, every prediction request will pass 
    # through serve_tf_examples_fn().
    # With every request, we parse the serialized tf.Example records and apply the 
    # preprocessing steps to the raw request data.
    # The model then makes a prediction on the preprocessed data.
    
    # Define the model signature, which includes the serving function we will describe later.
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ),
    }
    
    model.save(
        fn_args.serving_model_dir, 
        save_format="tf", 
        signatures=signatures
    )

Overwriting ./module.py


In [30]:
# The Transform component wraps TensorFlow Transform (tf.Transform) to 
# preprocess data in a TFX pipeline.
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=_module_file
)

context.run(transform, enable_cache=True)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying download_dataset.py -> build/lib
copying module.py -> build/lib
installing to /var/folders/vb/glk9sq0x22qbdp2r6prn8h600000gn/T/tmpe0iunen5
running install
running install_lib
copying build/lib/download_dataset.py -> /var/folders/vb/glk9sq0x22qbdp2r6prn8h600000gn/T/tmpe0iunen5
copying build/lib/module.py -> /var/folders/vb/glk9sq0x22qbdp2r6prn8h600000gn/T/tmpe0iunen5
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info 

/Users/gb/Desktop/python-3.8.12/env/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./tfx_pipeline_output/customer_complaints/_wheels/tfx_user_code_Transform-0.0+553e8696532c2eb0fc54a3cc5459d47e0b3187736148a66488aae0e462aa9bc2-py3-none-any.whl
Processing ./tfx_pipeline_output/customer_complaints/_wheels/tfx_user_code_Transform-0.0+553e8696532c2eb0fc54a3cc5459d47e0b3187736148a66488aae0e462aa9bc2-py3-none-any.whl
Processing ./tfx_pipeline_output/customer_complaints/_wheels/tfx_user_code_Transform-0.0+553e8696532c2eb0fc54a3cc5459d47e0b3187736148a66488aae0e462aa9bc2-py3-none-any.whl


2022-07-27 20:57:54.302056: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [31]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}),
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}),
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}),
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}),
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}),
 'post_transform_schema': OutputChannel(arti

In [32]:
# Examine the transform_graph artifact. It points to a directory containing 3 subdirs.
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [33]:
!tree {PIPELINE_ROOT}/Transform

./tfx_pipeline_output/customer_complaints/Transform
├── post_transform_anomalies
│   └── 5
│       └── SchemaDiff.pb
├── post_transform_schema
│   └── 5
│       └── schema.pbtxt
├── post_transform_stats
│   └── 5
│       └── FeatureStats.pb
├── pre_transform_schema
│   └── 5
│       └── schema.pbtxt
├── pre_transform_stats
│   └── 5
│       └── FeatureStats.pb
├── transform_graph
│   └── 5
│       ├── metadata
│       │   └── schema.pbtxt
│       ├── transform_fn
│       │   ├── assets
│       │   │   ├── vocab_compute_and_apply_vocabulary_1_vocabulary
│       │   │   ├── vocab_compute_and_apply_vocabulary_2_vocabulary
│       │   │   ├── vocab_compute_and_apply_vocabulary_3_vocabulary
│       │   │   ├── vocab_compute_and_apply_vocabulary_4_vocabulary
│       │   │   └── vocab_compute_and_apply_vocabulary_vocabulary
│       │   ├── saved_model.pb
│       │   └── variables
│       │       ├── variables.data-00000-of-00001
│       │       └── variables.index
│       └── transformed_meta

In [34]:
# The transformed_metadata subdir contains the schema of the preprocessed data.
# The transform_fn subdir contains the actual preprocessing graph.
# The metadata subdir contains the schema of the original data.

# Take a look at some transformed examples.

# Get the URI of the output artifact representing the transformed examples, which is a directory.
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')
print(train_uri)
HR()

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
print(tfrecord_filenames)
HR()

# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
print(dataset)
HR()

# Iterate over the first 1 records and decode them.
# for tfrecord in dataset.take(1):
#     example = tf.train.Example()
#     example.ParseFromString(tfrecord.numpy())
#     print(example)

./tfx_pipeline_output/customer_complaints/Transform/transformed_examples/5/Split-train
----------------------------------------
['./tfx_pipeline_output/customer_complaints/Transform/transformed_examples/5/Split-train/transformed_examples-00000-of-00001.gz']
----------------------------------------
<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>
----------------------------------------


In [35]:
# A Dataset comprising records from one or more TFRecord files.
type(dataset)

tensorflow.python.data.ops.readers.TFRecordDatasetV2

In [36]:
for raw_record in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    for x in example.features.feature:
        print(x)  

company_response_xf
state_xf
zip_code_xf
consumer_complaint_narrative_xf
issue_xf
product_xf
consumer_disputed_xf
sub_product_xf


---
# 6. Model training
<a id='trainer'></a>
## 6.1 Trainer

https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Trainer

<a href="#top">[back to top]</a>

In [37]:
for raw_record in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    for x in example.features.feature:
        print(x)  

consumer_complaint_narrative_xf
sub_product_xf
product_xf
consumer_disputed_xf
issue_xf
zip_code_xf
company_response_xf
state_xf


In [38]:
for raw_record in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    #print(example)

In [39]:
trainer_file = _module_file   #os.path.join(os.getcwd(), '../components/module.py')

In [40]:
# The Trainer TFX pipeline component trains a TensorFlow model.

TRAINING_STEPS = 500 # 1000
EVALUATION_STEPS = 100

trainer = tfx.components.Trainer(
    module_file=trainer_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=tfx.proto.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=tfx.proto.EvalArgs(num_steps=EVALUATION_STEPS)
    
    # tfx.proto.trainer_pb2 now deprecated
    # train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    # eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS)
)

context.run(trainer, enable_cache=True)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying download_dataset.py -> build/lib
copying module.py -> build/lib
installing to /var/folders/vb/glk9sq0x22qbdp2r6prn8h600000gn/T/tmphbxj58xs
running install
running install_lib
copying build/lib/download_dataset.py -> /var/folders/vb/glk9sq0x22qbdp2r6prn8h600000gn/T/tmphbxj58xs
copying build/lib/module.py -> /var/folders/vb/glk9sq0x22qbdp2r6prn8h600000gn/T/tmphbxj58xs
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/folders/

/Users/gb/Desktop/python-3.8.12/env/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./tfx_pipeline_output/customer_complaints/_wheels/tfx_user_code_Trainer-0.0+553e8696532c2eb0fc54a3cc5459d47e0b3187736148a66488aae0e462aa9bc2-py3-none-any.whl
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 consumer_complaint_narrative_x  [(None, 1)]         0           []                               
 f (InputLayer)                                                                                   
                                                                                                  
 tf.reshape (TFOpLambda)        (None,)              0           ['consumer_complaint_narrative_xf
                                                                 [0][0]']                         
                                                                                                  
 keras_layer (KerasLayer

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [41]:
!tree {PIPELINE_ROOT}/Trainer

./tfx_pipeline_output/customer_complaints/Trainer
├── model
│   └── 6
│       ├── Format-Serving
│       │   ├── assets
│       │   │   ├── vocab_compute_and_apply_vocabulary_1_vocabulary
│       │   │   ├── vocab_compute_and_apply_vocabulary_2_vocabulary
│       │   │   ├── vocab_compute_and_apply_vocabulary_3_vocabulary
│       │   │   ├── vocab_compute_and_apply_vocabulary_4_vocabulary
│       │   │   └── vocab_compute_and_apply_vocabulary_vocabulary
│       │   ├── keras_metadata.pb
│       │   ├── saved_model.pb
│       │   └── variables
│       │       ├── variables.data-00000-of-00001
│       │       └── variables.index
│       └── logs
│           ├── train
│           │   └── events.out.tfevents.1658923313.Georges-MacBook-Air.local.2860.0.v2
│           └── validation
│               └── events.out.tfevents.1658923677.Georges-MacBook-Air.local.2860.1.v2
└── model_run
    └── 6

10 directories, 11 files


### Load TensorBoard (how useful is this??)

In [42]:
# model_artifact_dir = trainer.outputs['model'].get()[0].uri

In [43]:
# log_dir = os.path.join(model_artifact_dir, 'logs/')

# %load_ext tensorboard
# %tensorboard --logdir {log_dir} 

# 7. Model analysis and validation
<a id='resolver'></a>
## 7.1 Resolver

https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/dsl/Resolver

<a href="#top">[back to top]</a>

In [44]:
# Resolver is a special TFX node which handles special artifact resolution 
# logics that will be used as inputs for downstream nodes.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          # Note: .with_id() is a method of tfx.dsl.Resolver. 
          # This replaces the deprecated `instance_name` in TFX 0.30.0
          type=tfx.types.standard_artifacts.ModelBlessing
      ),
).with_id('latest_blessed_model_resolver')

context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [45]:
model_resolver.id

'latest_blessed_model_resolver'

<a id='evaluator'></a>
## 7.2 Evaluator
<a href="#top">[back to top]</a>

https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Evaluator

In [46]:
# Note: it always blesses on first run even if below threshold

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name="serving_default",
            label_key="consumer_disputed",
            # preprocessing_function_names=["transform_features"],
        )
    ],
    
    slicing_specs=[
        tfma.SlicingSpec(), 
        tfma.SlicingSpec(feature_keys=["product"])
    ],
    
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            # BinaryAccuracy must be over 0.65
                            lower_bound={"value": 0.65}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            # Validate if BinaryAccuracy is at least 0.01 higher than baseline model.
                            absolute={'value': 0.01} # -1e-10
                        )
                    ),
                ),
                tfma.MetricConfig(class_name="ExampleCount"),
                # tfma.MetricConfig(class_name="Precision"),
                # tfma.MetricConfig(class_name="Recall"),
                # tfma.MetricConfig(class_name="AUC"),
            ],
        )
    ],
)


In [ ]:
# The Evaluator TFX pipeline component performs deep analysis on the 
# training results for your models.
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
)

context.run(evaluator, enable_cache=True)

In [ ]:
evaluator.outputs

In [ ]:
context.show(evaluator.outputs['evaluation'])

In [ ]:
!tree {PIPELINE_ROOT}/Evaluator

In [ ]:
# To see the visualization for sliced evaluation metrics, 
# directly call the TensorFlow Model Analysis library.

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# GB: CURRENTLY BROKEN!
# Show data sliced along feature column trip_start_hour.
# tfma.view.render_slicing_metrics(
#     tfma_result, slicing_column='trip_start_hour')

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

In [ ]:
# Also verify the success by loading the validation result record.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

<a id='pusher'></a>
## 7.3 Pusher
<a href="#top">[back to top]</a>

The Pusher component is usually at the end of a TFX pipeline. It checks whether a model has passed validation, and if so, exports the model to _serving_model_dir.


In [ ]:
# The Pusher component is used to push a validated model to a 
# deployment target during model training or re-training.

#from tfx.proto import pusher_pb2
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR
        )
    )
)

context.run(pusher, enable_cache=True)

In [ ]:
# Examine the output artifacts of Pusher
pusher.outputs

In [ ]:
# The Pusher will export your model in the SavedModel format, which looks like this:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
print(push_uri)
HR()

model = tf.saved_model.load(push_uri)
print(model)
HR()

for item in model.signatures.items():
    #pp.pprint(item)
    print(item)

In [ ]:
!tree {SERVING_MODEL_DIR}

---
# 8. Model deployment
<a id='model_server'></a>
## 8.1 Model server
<a href="#top">[back to top]</a>

https://github.com/tensorflow/serving/blob/master/tensorflow_serving/tools/docker/Dockerfile

In [ ]:
def tfserver_colab(SERVING_MODEL_DIR):
    import os
    import subprocess

    # If on Colab, cannot run Docker, so need to install manually
    !echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
    curl -s https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
    !apt -qq update
    !apt-get -qq install tensorflow-model-server

    HR()
    
    # https://docs.python.org/3/library/subprocess.html
    colab_script=(f"nohup \
        tensorflow_model_server \
        --rest_api_port=8501 \
        --model_name=my_model \
        --model_base_path={SERVING_MODEL_DIR} > server.log 2>&1 &")

    # Use stdout=PIPE to have Python redirect the data from the process to a new file handle, which can be accessed through result.stdout
    #result = subprocess.Popen(colab_script, shell=False)

    # Since we set the shell argument to True, this is the process ID of the spawned shell.
    # But we really want to PID of the server -- how to fix this?
    # https://docs.python.org/2/library/subprocess.html#subprocess.Popen.pid

    #result = subprocess.Popen(colab_script, shell=True)
    result = subprocess.run(colab_script, shell=True) 
    print("Running Tensorflow Server on Colab.")
    print(f"result: {result}")
    print(f"returncode: {result.returncode}")

    if result.returncode == 0:
        print("Tensorflow Server started successfully.")


In [ ]:
def tfserver_docker(SERVING_MODEL_DIR):
    import subprocess

    # Reference this example:
    # https://www.tensorflow.org/tfx/serving/docker

    # Upon container stop, remove container via --rm
    docker_script=(f"docker run --rm -t -p 8500:8500 -p 8501:8501 \
        --name tfserving_model \
        --mount type=bind,source={SERVING_MODEL_DIR},target=/models/my_model \
        -e MODEL_NAME=my_model \
        tensorflow/serving:latest &")

    result = subprocess.run(docker_script, shell=True)

    print("Running Tensorflow Server locally via Docker")
    #print(f"result: {result}")
    print(f"return code: {result.returncode}")

    if result.returncode == 0:
        print("Tensorflow Server started successfully.")

In [ ]:
import sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    tfserver_colab(SERVING_MODEL_DIR)
    HR()
    !tail server.log
else:
    tfserver_docker(SERVING_MODEL_DIR)
    HR()

In [ ]:
# Introspection and check
import os
import glob

latest_model = max(glob.glob(os.path.join(SERVING_MODEL_DIR, '*/')), key=os.path.getmtime)
print(latest_model)
HR()

print("Show all tag-sets in a SavedModel:\n")
!saved_model_cli show --dir {latest_model}
HR()

print("Show all available SignatureDef keys in the MetaGraphDef specified by tag-set 'serve':\n")
!saved_model_cli show --dir {latest_model} --tag_set serve
HR()

# print("Show all inputs and outputs TensorInfo for the specific SignatureDef 'serving_default' in the MetaGraph:")
# print("Note that shape: (-1) implies 1-D shape for inference data, eg {'examples': [{...}]}\n")
# !saved_model_cli show --dir {latest_model} --tag_set serve --signature_def serving_default

In [ ]:
def control_tfserver_docker():
    
    print("Controlling Tensorflow Server via Docker")
    tfc_container = 'tfserving_model'
    HR()
    
    # https://docker-py.readthedocs.io/en/stable/client.html
    client = docker.from_env()
    print(f"List of containers: {client.containers.list()}")
    
    # Search for this container by name, `tfserving_model`
    client_tfs = client.containers.get(tfc_container)    
    print(f"Status of tfserving_model container: {client_tfs.status}")
    HR()
    
    # Check to see we can stop this container
    #print("Stopping container")
    print(f"Stopping container {tfc_container}\n")
    client_tfs.stop(timeout=5)
        
    # Best way to check is to try to re-init new instance
    client = docker.from_env()
    print(f"List of containers: {client.containers.list()}")
    HR()
    
    try:
        client_tfs = client.containers.get(tfc_container)
    except:
        print(f"Container {tfc_container} not found.")
    
    HR()
    
    print("In order to start up, need to call tfserver_docker() again.")

    tfserver_docker(SERVING_MODEL_DIR)
    
    # Hack for now. Should learn proper Python mechanism for waiting for response
    !sleep 5
    HR()
    
    client = docker.from_env(timeout=2)
    print(f"List of containers: {client.containers.list()}")
    
    # Search for this container by name, `tfserving_model`
    client_tfs = client.containers.get(tfc_container)    
    print(f"Status of tfserving_model container: {client_tfs.status}")
    HR()
    !docker ps

In [ ]:
def control_tfserver_colab():
    
    # Find all pids of Tensorflow Server
    def get_process_by_name(name):
        import re, psutil

        pid_array = []
        for proc in psutil.process_iter():
            if hasattr(proc, 'name') and proc.status != 'terminated':
                if re.match(".*" + name + ".*", proc.name()):
                    pid_array.append(proc)
        return pid_array

    tfs_pid = get_process_by_name('tensorflow_model_server')
    print(tfs_pid)

    HR()

    if tfs_pid:   
        for x in tfs_pid:
            print(x.pid)
            print(f"Killing TensorFlow Server process {x.pid}")
            !kill {x.pid}
    else:
        print("TFS is not running.")

    !sleep 3
    HR()
    !ps
    HR()
    
    print("Restarting TensorFlow Server process on Colab.")
    tfserver_colab(SERVING_MODEL_DIR)
    !sleep 5
    !ps
        

In [ ]:
if IN_COLAB:
    control_tfserver_colab()
else:
    control_tfserver_docker()

<a id='client_request_simple'></a>
## 8.2 Client request (simple)
<a href="#top">[back to top]</a>

In [ ]:
# Python client for TensorFlow Model Server
def client_simple():

    print("=============== START ===============")

    # 1. Taken from: https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tft
    #
    # 2. There are three base Feature types:
    #   tf.train.BytesList
    #   tf.train.FloatList
    #   tf.train.Int64List
    #
    # 3. Fundamentally, a tf.train.Example is a {"string": tf.train.Feature} mapping.

    # TensorFlow Serving expects the input data as a JSON data structure
    feature = {
        'company':          tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Encore Capital Group'])),
        'company_response': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Closed with explanation'])),
        'consumer_complaint_narrative': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'I was denied employment because of a judgment against me. I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC . Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement. I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC.'])),
        'issue':            tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Disclosure verification of debt'])),
        'product':          tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Debt collection'])),
        'timely_response':  tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Yes'])),
    }

    # Create a Features message using tf.train.Example.
    # The tf.train.Example message (or protobuf) is a flexible message type that represents a 
    # {"string": value} mapping. It is designed for use with TensorFlow and is used throughout TFX.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    example_proto = example_proto.SerializeToString()

    json_data = {
        "signature_name": "serving_default",
        #"inputs": {"examples": {"b64": base64.b64encode(example_proto).decode('utf-8')}} # scalar shape
        "inputs": {"examples": [{"b64": base64.b64encode(example_proto).decode('utf-8')}]} # 1-D 
    }

    #print(type(json_data))
    # <class 'dict'>
    #HR()

    print("json_data:")
    print(json_data)
    # {'signature_name': 'serving_default', 'inputs': {'examples': [{'b64': 'CnMKGwoPY3VsbWVuX2RlcHRoX21tEggSBgoEzcyAQQocChBjdWxtZW5fbGVuZ3RoX21tEggSBgoEmplHQgodChFmbGlwcGVyX2xlbmd0aF9tbRIIEgYKBAAAVUMKFwoLYm9keV9tYXNzX2cSCBIGCgQAwKhF'}]}}

    HR()

    model_server_url = 'http://localhost:8501/v1/models/my_model:predict'
    headers = {"content-type": "application/json"}
    resp = requests.post(model_server_url, json=json_data, headers=headers)
    print("resp.json():")
    print(resp.json())
    HR()
    print("resp:")
    print(resp)

    print("=============== END ===============")
    
    

try:
    client_simple()
except Exception as e:
    print(f"Error: {e}")

<a id='client_request_complex'></a>
## 8.3 Client request (complex)
<a href="#top">[back to top]</a>

In [ ]:
# More complicated request
def client_complex():

    print("=============== START ===============")

    def serialize_example(feature_dict):
        """Create a serialized tf.example from feature dictionary."""

        # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/saved_model_cli.py#L597
        # Create a tf.train.Example-compatible data type.
        # This is a mostly-normalized data format for storing data for training and inference.
        # This contains a key-value store features where each key (string) maps to a tf.train.Feature message
        # Note that tf.train.Example message is just a wrapper around the Features message.
        # https://www.tensorflow.org/api_docs/python/tf/train/Example
        # https://www.tensorflow.org/tutorials/load_data/tfrecord
        # https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tft
        example = {}

        for feature_name, feature_list in feature_dict.items():
            if not isinstance(feature_list, list):
                raise ValueError(
                    f"feature value must be a list, but {feature_name}: {feature_list} is {type(feature_list)}"
                )

            # There are three base Feature types: tf.train.BytesList,tf.train.FloatList, tf.train.Int64List
            if isinstance(feature_list[0], float):
                example[feature_name] = (
                    tf.train.Feature(
                        float_list=tf.train.FloatList(value=[feature_list[0]])
                    ),
                )

            elif isinstance(feature_list[0], str):
                # feature_list[0] = feature_list[0].numpy()
                example[feature_name] = (
                    tf.train.Feature(
                        bytes_list=tf.train.BytesList(value=[feature_list[0]])
                    ),
                )

            elif isinstance(feature_list[0], bytes):
                # feature_list[0] = feature_list[0].numpy()
                example[feature_name] = (
                    tf.train.Feature(
                        bytes_list=tf.train.BytesList(value=[feature_list[0]])
                    ),
                )

            # https://www.tensorflow.org/tutorials/load_data/tfrecord
            # elif isinstance(feature_list[0], type(tf.constant(0))):
            #   feature_list[0] = feature_list[0].numpy() # BytesList won't unpack a string from an EagerTensor.
            #   example[feature_name] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[feature_list[0]])),

            elif isinstance(feature_list[0], int):
                example[feature_name] = (
                    tf.train.Feature(
                        int64_list=tf.train.Int64List(value=[feature_list[0]])
                    ),
                )

            else:
                raise ValueError(
                    f"Type {type(feature_list[0])} for value {feature_list[0]} is not supported in this operation"
                )

        # Create a Features message using tf.train.Example
        example_proto = tf.train.Example(features=tf.train.Features(feature=example))
        return example_proto.SerializeToString()


    # -----------------------------------------------

    # Example: {"a": False, "b": 4, "c": b'goat', "d": 0.9876}

    feature_dict = {
        "company": [b'Encore Capital Group'],
        "company_response": [b'Closed with explanation'],
        "consumer_complaint_narrative": [b'I was denied employment because of a judgment against me. I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC . Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement. I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC.'],
        "issue": [b'Disclosure verification of debt'],
        "product": [b'Debt collection'],
        "timely_response": [b'Yes']
    }

    
    
# 'company':          tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Encore Capital Group'])),
# 'company_response': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Closed with explanation'])),
# 'consumer_complaint_narrative': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'I was denied employment because of a judgment against me. I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC . Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement. I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC.'])),
# 'issue':            tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Disclosure verification of debt'])),
# 'product':          tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Debt collection'])),
# 'timely_response':  tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Yes'])),

    
    example_proto = serialize_example(feature_dict)

    # Testing: decode the message
    example_proto_decoded = tf.train.Example.FromString(example_proto)
    print(example_proto_decoded)
    HR()

    # -----------------------------------------------

    # Transform into JSON-compatible format
    # Apply base64.b64encode to base64 encode the bytes, then use decode('utf-8')
    # to create Base64 encoded data using human-readable characters.
    json_data = {
        "signature_name": "serving_default",
        "inputs": {
            "examples": [{"b64": base64.b64encode(example_proto).decode("utf-8")}]
        }  # 1-D
        # "inputs": {"examples": {"b64": base64.b64encode(example_proto).decode('utf-8')}} # scalar shape
    }

    print(json_data)
    # {'signature_name': 'serving_default', 'inputs': {'examples': [{'b64': 'CnMKFwoLYm9keV9tYXNzX2cSCBIGCgQAYGpFChwKEGN1bG1lbl9sZW5ndGhfbW0SCBIGCgTNzOhBCh0KEWZsaXBwZXJfbGVuZ3RoX21tEggSBgoEmhk1QwobCg9jdWxtZW5fZGVwdGhfbW0SCBIGCgSamZVB'}]}}

    HR()

    # -----------------------------------------------

    model_server_url = "http://localhost:8501/v1/models/my_model:predict"
    headers = {"content-type": "application/json"}
    resp = requests.post(model_server_url, json=json_data, headers=headers)

    print(f"resp.json(): {resp.json()}")
    HR()
    print(f"resp: {resp}")

    print("=============== END ===============")

    

try:
    client_complex()
except Exception as e:
    print(f"Error: {e}")

<a id='client_request_metadata'></a>
## 8.4 Client request (metadata)
<a href="#top">[back to top]</a>

In [ ]:
def client_metadata():
    url = "http://localhost:8501/v1/models/my_model/metadata"
    print(url)
    HR()
    response = requests.get(url=url) 
    print(response.text)
    
try:
    client_metadata()
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Check entire project structure
!tree -I __pycache__

In [ ]:
# Check total size of this project
!du -sch

In [ ]:
# Find biggest 10 folders/files at top level
!du -hs * | sort -rh | head -10

In [ ]:
# Find largest directories, inner level
!du -ha | sort -n -r | head -n 20